### Problem Description:

Write a program that reads a text file that contains 0's and 1's in a regular N x M sized grid.  Here is an example of what a valid input file looks like:

0110 <br>
0100 <br>
0010 <br>
0011 <br>

After reading the file, your goal is to find the number of "connected shapes" in the data.  A connected shape is defined as a group of 1's that are immediately adjacent to each other.  The 1's must be immediately to the left, right, top, or bottom of each other to count as connected (diagonals do not count as connected).

In the example above, your program would return 2, because there are two connected shapes: <br>
11 <br>
1<br>
at position (0, 1) and <br> 
1 <br>
11<br>
at position (2, 2)

You only need to return the number of connected shapes that you've found.  You do not need to return the position of the shapes.

## The above prompt is exempt from the license below and was provided by Maestro for the purpose of evaluation for employment.
## Anything below this is the work of and property of David Gabriel

## Notes

A brief video explaining the solution.
https://drive.google.com/file/d/1rq0U3h8kZYhOEYDgCEWrZup8PpEBbg9U/view

#### Key Assumptions
At first, as always I wanted to explicitly state the assumptions and define any tricky edge cases before coding. The following assumptions were made in the completion of this task. 
- Single 1's are a "group" or "shape". 
- Only "valid" inputs allowed. That means for instance, we will ignore possible cases where, maybe a list, or an escape character is inserted where a single 0 or 1 is expected. Also, only 0's and 1's in rectangular grid.

#### Goals
The goal is to solve the problem as stated, but also to do so with consideration towards: <br>
-Accuracy in all cases<br>
-Minimum algorithmic complexity<br>
-Possibility to parallelize the algorithm and distribute parts of the workload<br>


##### Mapping Stage:
This approach is to scan all the lines one at a time, detecting 1's. When a 1 is detected,  We check for adjectent known shapes that it would be connected to according to the prompt. If we see that an adjacent shape is detected, we add this 1 to that shape. Shapes are tracked in a hash map as keys. The idea there is to potentially reduce the complexity to check if a 1 is a shape member (O(1) instead of O(N) where N is the number of shape members). This was not fully implemented in this version due to time constraints. 

Now this is all well and good, but there's a catch. When we scan into a shape, it is possible we see a shape member, but have not yet scanned its neighboring shape members. In this case, we detect this as a new shape. The problem is we end up creating two adjacent shapes that actually should be connected. 
The mapping phase has complexity O(MxN).

##### Reduce Stage:
In order to solve the problem of detecting a new shape, because its adjecent members are unknown, we apply a reduction. We iterate over the shapes we have detected, and look for connected shapes in order to join them. This join process has complexity O(4P^2) where P is the number of shape members in the space. The four is because it checks in 4 dimensions. In different dimensionality spaces this would change accordingly. 

##### Parallelizability 
The mapping stage is embarassingly parallel, if using something like a dataframe to store indices. If sharding the space, we may need an iterative reducer, depending on the sharding type. This could conceivably be a very scalable solution, especially for sparse spaces. 


##### Boundary Walk (another possible solution)
So, I began with the concept that I would instantiate a 2d cursor and use iterators to search for the first 1 in the matrix. Pretty standard idea, but not the only approach. Then what we do next once the shape is first detected is the most important part of this algorithm design. I investigated some concepts related to boundary traversal. That is walking along the perimeter of the shape and determining an outer edge of the shape. I thought this could possibly reduce number of array accesses. However, in praticality such a walking algorithm is complex. Something like a Tremeau walk, but modified for boundary traversal might work. Unfortunately for many shapes present in the small data set, this actually tends to increase the the number of access, to determine the correct path around the shape. Additionally, this algorithm stage starts to accumulate state in order to make decisions around where to make the next step, avoid looping inside the shape, etc. Less than ideal for this kata solution. 
There may be some neural net implementation of this possible that would be highly efficient. Especially for spaces with large area to perimeter ratios. This could have complexity of O(MxN)-c for a space of size MXN, where c is the number of internal 1's which a stateful boundary walk doesn't need to scan. An interesting concept but too complicated to optimize for this kata.

Next, I tried another approach which is what I decided to go with. This is a two stage approach. Map, and reduce.

##### Final Thoughts:
This is a fun and interesting problem. I think it is a good kata to demonstrate algorithmic problem solving. However, a full implementation takes a bit longer than typical interview kata often do. At least that is the case for me.

I found the number of shapes to be 14 for the data_small.txt and 573 for the data_large.txt. 

I honestly have trouble counting the shapes in the small data set without printing it out and using crayons to highlight the boundaries. I no longer own a printer or crayons. Therefore I am not certain if I have an off by one error or something like that. I learned that crayon trick as a junior electrical engineer, as a way to highlight complex circuit groups in a less headache inducing way. Also, it like they say, "there's only 2 hard problems in computer science: naming things, cache invalidation, and off by one errors." 

I hope you enjoyed reading my solution to this kata. I enjoyed producing it! 

##### Also...
I added an Apache license to this and plan to share it on my github. Considering I did this for free that is more than fair in my opinion. If you differ in your opinion, please email me, the sole contributor to this solution. My contact is:
David Gabriel
contact@david-gabriel.com

In [1]:
# Assumptions: single 1's are a "group" or "shape". 
# Only "valid" inputs allowed. 

In [2]:
#preprocess data

#read in raw data
with open('data_small.txt') as f:
    lines = f.readlines()


array = []
for line in lines:
    line = line.replace('\n', '')
    #convert to ints to avoid math headaches with strings
    line = list(map(int, line))
    array.append(list(line))


In [3]:
class connectedShapes:
    def __init__(self, array):
        #no validators for now, because this is a demo
        self.array = array
        self.yAxisLen = len(array)
        self.xAxisLen = len(array[1])
        self.shapeMembers = {}
        self.shapes = []
        self.reducedShapes = []
        self.reducing = True

        print("Loaded data. y axis length: " + str(self.yAxisLen) + " , x axis length: " + str(self.xAxisLen))

    def printArray(self):
        line = ""
        for idx, i in enumerate(array):
            print(line)
            line = ""
            for jdx, j in enumerate(i):
                line = line + str(j)   
        print(line)

    def pintSizeOfArray(self):
        print("y axis size: " + str(len(array)))
        print("x axis size: " + str(len(array[1])))

    def cursorToString(self, idx,jdx):
        return str(idx) + ":" + str(jdx)

    #return index of known shape, or create new shape
    def checkIfLocationIsInAShape(self,idx,jdx):
        string = self.cursorToString(idx, jdx)
        for index, shape in enumerate(self.shapes):
            if string in shape.keys():
                return index
        return -1

    #when we find a 1, see if it is in a known shape, otherwise add it to a new shape
    def addMemberToShape(self,idx,jdx):
        #check all cardinal directions for adjecent 1.
        #if we find one, see if it is a known shape, and add id
            here = self.cursorToString(idx,jdx)
            try:
                if (self.array[idx][jdx+1] == 1):
                    string = self.cursorToString(idx, jdx+1)
                    for index, shape in enumerate(self.shapes):
                        if string in shape.keys():
                            self.shapes[index].update({here:None})
                            return
            except IndexError:
                pass

            try:
                if (self.array[idx-1][jdx] == 1):
                    string = self.cursorToString(idx-1, jdx)
                    for index, shape in enumerate(self.shapes):
                        if string in shape.keys():
                            self.shapes[index].update({here:None})  
                            return
            except IndexError:
                pass

            try:
                if (self.array[idx][jdx-1] == 1):
                    string = self.cursorToString(idx, jdx-1)
                    for index, shape in enumerate(self.shapes):
                        if string in shape.keys():
                            self.shapes[index].update({here:None})  
                            return
            except IndexError:
                pass
                
            try:
                if (self.array[idx+1][jdx] == 1):
                    string = self.cursorToString(idx+1, jdx)
                    for index, shape in enumerate(self.shapes):
                        if string in shape.keys():
                            self.shapes[index].update({here:None})
                            return
            except IndexError:
                pass
                
            self.shapes.append({here:None})
            return

    ## check for adjecent shapes, and merge them. idempotent, for iterating in a loop. not guarunteed to merge all first pass.
    def reduceShapes(self, shapes):
        #local
        reducedShapes = shapes
        
        self.reducing = False
       
        for shape in self.shapes:
            for key in shape.keys():
                cursor = key.split(":")
                idx = int(cursor[0])
                jdx = int(cursor[1])
                for joinShape in self.shapes:
                    if shape != joinShape:
                        for joinKey in joinShape.keys():
                            #check if shapes have adjecent members
                            if (joinKey == self.cursorToString(idx,jdx+1) or joinKey == self.cursorToString(idx +1,jdx) or joinKey == self.cursorToString(idx,jdx-1) or joinKey == self.cursorToString(idx -1,jdx)):
                                #merge dicts if match
                                newShape = shape | joinShape
                                #see if shape already exists before adding to reduced shapes
                                if newShape not in reducedShapes:
                                    reducedShapes.append(newShape) 
                                    self.reducing = True
                          
        return  reducedShapes 
        
    # scan through array, check if 1's are in known shape, or create a new shape. 
    #reduce adjacent shapes, because leading edges are tricky
    def traverseArray(self):
        for idx, i in enumerate(self.array):
            for jdx, j in enumerate(i):
                if (j == 1):
                    index = self.checkIfLocationIsInAShape(idx,jdx)
                    if index == -1:
                        self.addMemberToShape(idx,jdx)
                        

                    
    def solveArray(self):
        #map
        self.traverseArray()
        self.reducedShapes = self.shapes
        #reduce
        shapes.reducedShapes =[]
        shapes.reducedShapes = shapes.reduceShapes(shapes.reducedShapes)
        #final result
        print("The number of shapes is: "+str(len(self.reducedShapes)))


In [4]:
shapes = connectedShapes(array)

shapes.printArray()

Loaded data. y axis length: 23 , x axis length: 65

00000000000000000000000000000000000000000000000000000000000000000
00000000001110001110000001110000000000000111111100000000000000000
00000000001111011111100000011000000000000000111111111110000000000
00000000000111000011000000011111111111111111111100000000000000000
01100110000001100001111000001111111111100111000110000001111100000
00111110000000000000111100001110000000000000000000000000111111111
00010000000000000011111110000000000000000011111000000000000010000
00111111111111111111000000000000000000000000111000000000011111000
00000111000000111000000000000000000000000001111111100000000011111
00000000000111111111000000000000000000000111111100000000000000110
00000000000000000000000000000000000000000000011000000000000011111
00000000000111110000011111100000000111110000001111100000000000000
00000000111000111111111000000000000000010000000000000000000000000
00000000000000000000001111111111111111111000000000000000001111111
000000000001110011100000

In [5]:
shapes.solveArray()

The number of shapes is: 14


In [6]:
#preprocess data

#read in raw data
with open('data_large.txt') as f:
    lines = f.readlines()


array = []
for line in lines:
    line = line.replace('\n', '')
    #convert to ints to avoid math headaches with strings
    line = list(map(int, line))
    array.append(list(line))

shapes = connectedShapes(array)
shapes.solveArray()

Loaded data. y axis length: 99 , x axis length: 99
The number of shapes is: 573


In [7]:
#reducer debugging code (could be good test cases!)

In [8]:
s1 = {'1:16': None, '1:17': None, '1:18': None, '2:16': None, '2:17': None, '2:18': None, '2:19': None, '2:20': None, '3:18': None, '3:19': None, '4:19': None, '4:20': None, '4:21': None, '4:22': None, '5:20': None, '5:21': None, '5:22': None, '5:23': None, '6:20': None, '6:21': None, '6:22': None, '6:23': None, '6:24': None}
s2 = {'2:15': None}

In [9]:
shapes.shapes =[]
shapes.reducedShapes=[]
shapes.shapes.append(s1)
shapes.shapes.append(s2)
print(shapes.shapes)

[{'1:16': None, '1:17': None, '1:18': None, '2:16': None, '2:17': None, '2:18': None, '2:19': None, '2:20': None, '3:18': None, '3:19': None, '4:19': None, '4:20': None, '4:21': None, '4:22': None, '5:20': None, '5:21': None, '5:22': None, '5:23': None, '6:20': None, '6:21': None, '6:22': None, '6:23': None, '6:24': None}, {'2:15': None}]


In [10]:
shapes.reducing = True
while shapes.reducing:
            shapes.reducedShapes = shapes.reduceShapes(shapes.reducedShapes)
print(len(shapes.reducedShapes))

1


In [11]:
print(shapes.reducedShapes)

[{'1:16': None, '1:17': None, '1:18': None, '2:16': None, '2:17': None, '2:18': None, '2:19': None, '2:20': None, '3:18': None, '3:19': None, '4:19': None, '4:20': None, '4:21': None, '4:22': None, '5:20': None, '5:21': None, '5:22': None, '5:23': None, '6:20': None, '6:21': None, '6:22': None, '6:23': None, '6:24': None, '2:15': None}]


              Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding shares, or (iii) beneficial ownership of such entity.

      "You" (or "Your") shall mean an individual or Legal Entity
      exercising permissions granted by this License.

      "Source" form shall mean the preferred form for making modifications,
      including but not limited to software source code, documentation
      source, and configuration files.

      "Object" form shall mean any form resulting from mechanical
      transformation or translation of a Source form, including but
      not limited to compiled object code, generated documentation,
      and conversions to other media types.

      "Work" shall mean the work of authorship, whether in Source or
      Object form, made available under the License, as indicated by a
      copyright notice that is included in or attached to the work
      (an example is provided in the Appendix below).

      "Derivative Works" shall mean any work, whether in Source or Object
      form, that is based on (or derived from) the Work and for which the
      editorial revisions, annotations, elaborations, or other modifications
      represent, as a whole, an original work of authorship. For the purposes
      of this License, Derivative Works shall not include works that remain
      separable from, or merely link (or bind by name) to the interfaces of,
      the Work and Derivative Works thereof.

      "Contribution" shall mean any work of authorship, including
      the original version of the Work and any modifications or additions
      to that Work or Derivative Works thereof, that is intentionally
      submitted to Licensor for inclusion in the Work by the copyright owner
      or by an individual or Legal Entity authorized to submit on behalf of
      the copyright owner. For the purposes of this definition, "submitted"
      means any form of electronic, verbal, or written communication sent
      to the Licensor or its representatives, including but not limited to
      communication on electronic mailing lists, source code control systems,
      and issue tracking systems that are managed by, or on behalf of, the
      Licensor for the purpose of discussing and improving the Work, but
      excluding communication that is conspicuously marked or otherwise
      designated in writing by the copyright owner as "Not a Contribution."

      "Contributor" shall mean Licensor and any individual or Legal Entity
      on behalf of whom a Contribution has been received by Licensor and
      subsequently incorporated within the Work.

   2. Grant of Copyright License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      copyright license to reproduce, prepare Derivative Works of,
      publicly display, publicly perform, sublicense, and distribute the
      Work and such Derivative Works in Source or Object form.

   3. Grant of Patent License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      (except as stated in this section) patent license to make, have made,
      use, offer to sell, sell, import, and otherwise transfer the Work,
      where such license applies only to those patent claims licensable
      by such Contributor that are necessarily infringed by their
      Contribution(s) alone or by combination of their Contribution(s)
      with the Work to which such Contribution(s) was submitted. If You
      institute patent litigation against any entity (including a
      cross-claim or counterclaim in a lawsuit) alleging that the Work
      or a Contribution incorporated within the Work constitutes direct
      or contributory patent infringement, then any patent licenses
      granted to You under this License for that Work shall terminate
      as of the date such litigation is filed.

   4. Redistribution. You may reproduce and distribute copies of the
      Work or Derivative Works thereof in any medium, with or without
      modifications, and in Source or Object form, provided that You
      meet the following conditions:

      (a) You must give any other recipients of the Work or
          Derivative Works a copy of this License; and

      (b) You must cause any modified files to carry prominent notices
          stating that You changed the files; and

      (c) You must retain, in the Source form of any Derivative Works
          that You distribute, all copyright, patent, trademark, and
          attribution notices from the Source form of the Work,
          excluding those notices that do not pertain to any part of
          the Derivative Works; and

      (d) If the Work includes a "NOTICE" text file as part of its
          distribution, then any Derivative Works that You distribute must
          include a readable copy of the attribution notices contained
          within such NOTICE file, excluding those notices that do not
          pertain to any part of the Derivative Works, in at least one
          of the following places: within a NOTICE text file distributed
          as part of the Derivative Works; within the Source form or
          documentation, if provided along with the Derivative Works; or,
          within a display generated by the Derivative Works, if and
          wherever such third-party notices normally appear. The contents
          of the NOTICE file are for informational purposes only and
          do not modify the License. You may add Your own attribution
          notices within Derivative Works that You distribute, alongside
          or as an addendum to the NOTICE text from the Work, provided
          that such additional attribution notices cannot be construed
          as modifying the License.

      You may add Your own copyright statement to Your modifications and
      may provide additional or different license terms and conditions
      for use, reproduction, or distribution of Your modifications, or
      for any such Derivative Works as a whole, provided Your use,
      reproduction, and distribution of the Work otherwise complies with
      the conditions stated in this License.

   5. Submission of Contributions. Unless You explicitly state otherwise,
      any Contribution intentionally submitted for inclusion in the Work
      by You to the Licensor shall be under the terms and conditions of
      this License, without any additional terms or conditions.
      Notwithstanding the above, nothing herein shall supersede or modify
      the terms of any separate license agreement you may have executed
      with Licensor regarding such Contributions.

   6. Trademarks. This License does not grant permission to use the trade
      names, trademarks, service marks, or product names of the Licensor,
      except as required for reasonable and customary use in describing the
      origin of the Work and reproducing the content of the NOTICE file.

   7. Disclaimer of Warranty. Unless required by applicable law or
      agreed to in writing, Licensor provides the Work (and each
      Contributor provides its Contributions) on an "AS IS" BASIS,
      WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
      implied, including, without limitation, any warranties or conditions
      of TITLE, NON-INFRINGEMENT, MERCHANTABILITY, or FITNESS FOR A
      PARTICULAR PURPOSE. You are solely responsible for determining the
      appropriateness of using or redistributing the Work and assume any
      risks associated with Your exercise of permissions under this License.

   8. Limitation of Liability. In no event and under no legal theory,
      whether in tort (including negligence), contract, or otherwise,
      unless required by applicable law (such as deliberate and grossly
      negligent acts) or agreed to in writing, shall any Contributor be
      liable to You for damages, including any direct, indirect, special,
      incidental, or consequential damages of any character arising as a
      result of this License or out of the use or inability to use the
      Work (including but not limited to damages for loss of goodwill,
      work stoppage, computer failure or malfunction, or any and all
      other commercial damages or losses), even if such Contributor
      has been advised of the possibility of such damages.

   9. Accepting Warranty or Additional Liability. While redistributing
      the Work or Derivative Works thereof, You may choose to offer,
      and charge a fee for, acceptance of support, warranty, indemnity,
      or other liability obligations and/or rights consistent with this
      License. However, in accepting such obligations, You may act only
      on Your own behalf and on Your sole responsibility, not on behalf
      of any other Contributor, and only if You agree to indemnify,
      defend, and hold each Contributor harmless for any liability
      incurred by, or claims asserted against, such Contributor by reason
      of your accepting any such warranty or additional liability.

   END OF TERMS AND CONDITIONS

   APPENDIX: How to apply the Apache License to your work.

      To apply the Apache License to your work, attach the following
      boilerplate notice, with the fields enclosed by brackets "[]"
      replaced with your own identifying information. (Don't include
      the brackets!)  The text should be enclosed in the appropriate
      comment syntax for the file format. We also recommend that a
      file or class name and description of purpose be included on the
      same "printed page" as the copyright notice for easier
      identification within third-party archives.

   Copyright [yyyy] [name of copyright owner]

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.